# Seldon Kafka Integration Example with CIFAR10 Model

In this example we will run SeldonDeployments for a CIFAR10 Tensorflow model which take their inputs from a Kafka topic and push their outputs to a Kafka topic. We will experiment with both REST and gRPC Seldon graphs. For REST we will load our input topic with Tensorflow JSON requests and for gRPC we will load Tensorflow PredictRequest protoBuffers.

## Setup Kafka

Install Strimzi on cluster

In [ ]:
!helm repo add strimzi https://strimzi.io/charts/

In [ ]:
!helm install my-release strimzi/strimzi-kafka-operator

Set the following to whether you are running a local Kind cluster or a cloud based cluster.

In [ ]:
clusterType="kind"
#clusterType="cloud"

In [ ]:
if clusterType == "kind":
    !kubectl apply -f cluster-kind.yaml
else:
    !kubectl apply -f cluster-cloud.yaml

Get broker endpoint.

In [ ]:
if clusterType == "kind":
    res=!kubectl get service my-cluster-kafka-external-bootstrap -n default -o=jsonpath='{.spec.ports[0].nodePort}'
    port=res[0]
    %env BROKER=172.17.0.2:$port
else:
    res=!kubectl get service my-cluster-kafka-external-bootstrap -o=jsonpath='{.status.loadBalancer.ingress[0].hostname}'
    if len(res) == 1:
        hostname=res[0]
        %env BROKER=$h:9094
    else:
        res=!kubectl get service my-cluster-kafka-external-bootstrap -o=jsonpath='{.status.loadBalancer.ingress[0].ip}'
        ip=res[0]
        %env BROKER=$ip:9094

In [ ]:
!kubectl apply -f topics.yaml

## Install Seldon

In [ ]:
!kubectl create namespace seldon-system

In [ ]:
!helm install seldon-core seldon-core-operator \
    --repo https://storage.googleapis.com/seldon-charts \
    --set usageMetrics.enabled=true \
    --namespace seldon-system

[Follow our docs to connect to the metrics dashboard](https://docs.seldon.io/projects/seldon-core/en/latest/analytics/analytics.html).

In [ ]:
!helm install seldon-core-analytics seldon-core-analytics \
   --repo https://storage.googleapis.com/seldon-charts \
   --namespace seldon-system

## Download Test Request Data
We have two example datasets containing 50,000 requests in tensorflow serving format for CIFAR10. One in JSON format and one as length encoded proto buffers.

In [ ]:
!gsutil cp gs://seldon-datasets/cifar10/requests/tensorflow/cifar10_tensorflow.json.gz cifar10_tensorflow.json.gz
!gunzip cifar10_tensorflow.json.gz
!gsutil cp gs://seldon-datasets/cifar10/requests/tensorflow/cifar10_tensorflow.proto cifar10_tensorflow.proto

## Test CIFAR10 REST Model

Upload tensorflow serving rest requests to kafka. This may take some time dependent on your network connection.

In [ ]:
!python ../../../util/kafka/test-client.py produce $BROKER cifar10-rest-input --file cifar10_tensorflow.json

In [ ]:
res=!kubectl get service my-cluster-kafka-external-bootstrap -o=jsonpath='{.spec.clusterIP}'
ip=res[0]
%env BROKER_CIP=$ip

In [ ]:
!cat cifar10_rest.yaml | sed s/BROKER_IP/$BROKER_CIP:9094/ | kubectl create -f -

Looking at the metrics dashboard for Seldon you should see throughput we are getting. For a single replica on GKE with n1-standard-4 nodes we can see roughly 150 requests per second being processed.

![rest](tensorflow-rest-kafka.png)

In [ ]:
!kubectl delete -f cifar10_rest.yaml

## Test CIFAR10 gRPC Model

Upload tensorflow serving rest requests to kafka. This is a file of protobuffer `tenserflow.serving.PredictRequest` ([defn](https://github.com/tensorflow/serving/blob/master/tensorflow_serving/apis/predict.proto)). Each binary protobuffer is prefixed by the numbre of bytes. Out test-client python script reads them and sends to our topic. This may take some time dependent on your network connection.

In [ ]:
!python ../../../util/kafka/test-client.py produce $BROKER cifar10-grpc-input --file cifar10_tensorflow.proto --proto_name tensorflow.serving.PredictRequest

In [ ]:
res=!kubectl get service my-cluster-kafka-external-bootstrap -o=jsonpath='{.spec.clusterIP}'
ip=res[0]
%env BROKER_CIP=$ip

In [ ]:
!cat cifar10_grpc.yaml | sed s/BROKER_IP/$BROKER_CIP:9094/ | kubectl create -f -

Looking at the metrics dashboard for Seldon you should see throughput we are getting. For a single replica on GKE with n1-standard-4 nodes we can see around 220 requests per second being processed.

![rest](tensorflow-grpc-kafka.png)

In [ ]:
!kubectl delete -f cifar10_grpc.yaml